In [1]:
import math
import numpy as np
import random

# RSA

- RSA_setup(p,q):
- RSA_encrypt(m,e):
- RSA_decrypt(M,d):

In [2]:
def _ext_euc_alg(a, b):
    '''
    Performs Extended Euclidean Algorithm on two integers

            Parameters:
                    a : An integer
                    b : Another integer

            Returns:
                    (gcd, x, y) : tuple of GCD of (a,b) and (x,y) orthogonal to (a,b)
    '''
    if a == 0:
        return b, 0, 1
             
    gcd,x1,y1 = _ext_euc_alg(b % a, a) 
     
    # Update x and y using results of recursive call 
    x = y1 - (b // a) * x1 
    y = x1 
     
    return gcd,x,y

def _inv_mult(a, m):
    '''
    Find modular inverses using EEA

            Parameters:
                    a : Inverse to be found
                    m : Modulus – must have (a, m) == 1

            Returns:
                    inv : Modular inverse of a mod m
    '''
    gcd, x, _ = _ext_euc_alg(a, m)
    
    if gcd == 1:
        return (x + m) % m
    

def _mod_pow(a, e, m):
    '''
    Compute modular exponentation under modular arithmetic
    
            Parameters:
                    a : Base
                    e : Exponent
                    m : Modulus
            
            Returns:
                    b : Value of a^e % m
    '''
    a, b = a % m, 1
    
    if a == 0: return 0
    
    while e > 0:
        if (e & 1) == 1:
            b = (a * b) % m
        
        e >>= 1
        a = (a * a) % m
    
    return b


def RSA_setup(p,q):
    '''
    Sets up parameters for RSA algorithm

            Parameters:
                    p : Large prime
                    q : Another large prime

            Returns:
                    (n, e, d) : RSA modulus, public key, and private key
    '''
    n = p * q
    phi_n = (p - 1) * (q - 1)
    
    # select public key s.t. (e, phi_n) == 1
    found = False
    
    while not found:
        e = random.randint(1, phi_n)
        if math.gcd(e, phi_n) == 1:
            found = True
    
    d = _inv_mult(e, phi_n)
    
    return n, e, d


def RSA_encrypt(m, e, n):
    '''
    Encrypts messages with RSA algorithm

            Parameters:
                    m : Integer representation of message – must be between 1 and n
                    e : Public key – selected as per RSA_setup
                    n : Modulus – selected as per RSA_setup

            Returns:
                    M : Encrypted message
    '''
    return _mod_pow(m, e, n)


def RSA_decrypt(M, d, n):
    '''
    Decrypts messages with RSA algorithm

            Parameters:
                    M : Encrypted message – calculated as per RSA_encrypt
                    d : Private key – selected as per RSA_setup
                    n : Modulus – selected as per RSA_setup

            Returns:
                    m : Plaintext message
    '''
    return _mod_pow(M, d, n)

In [3]:
n, e, d = RSA_setup(17,19) # n = 323
print(f"Public Key: {e}\nPrivate Key: {d}\n")

m = 104 # Remeber, must be between 1 - n

M = RSA_encrypt(m, e, n)
print(f"Encrypted Message: {M}")

m1 = RSA_decrypt(M, d, n)
print(f"Decrypted Message: {m}")

Public Key: 247
Private Key: 7

Encrypted Message: 196
Decrypted Message: 104


In [33]:
class PairValEnc:
    def __init__(self, p, q):
        '''
        ()

                Parameters:
                        ()

                Returns:
                        ()
        '''
        n = p * q
        phi_n = (p - 1) * (q - 1)

        found = False

        while not found:
            e = random.randint(1, phi_n)
            if math.gcd(e, phi_n) == 1:
                found = True

        d = _inv_mult(e, phi_n)

        self.n = n
        self.phi_n = phi_n
        self.e = e
        self.d = d
        
        
    def load(self, values, mapping):
        '''
        ()

                Parameters:
                        ()

                Returns:
                        ()
        '''
        self.values = values
        self.mapping = mapping
        self.inv_mapping = {v:k for k, v in mapping.items()}
        
    def encrypt(self, x):
        '''
        ()

                Parameters:
                        ()

                Returns:
                        ()
        '''
        if x in self.values:
            return self._encrypt(self.mapping[x])
    
    def compare(self, x, y):
        '''
        ()

                Parameters:
                        ()

                Returns:
                        ()
        '''
        return self._cmp(x, y) == 1
        
    
    def decrypt(self, x, d):
        '''
        ()

                Parameters:
                        ()

                Returns:
                        ()
        '''
        if d == self.d:
            return self.inv_mapping[self._decrypt(x)]
    
    
    def _encrypt(self, x):
        '''
        ()

                Parameters:
                        ()

                Returns:
                        ()
        '''
        return _mod_pow(x, self.e, self.n)
    
    
    def _cmp(self, x, y):
        '''
        ()

                Parameters:
                        ()

                Returns:
                        ()
        '''
        return (x * y) % self.n
    
    
    def _decrypt(self, x):
        '''
        ()

                Parameters:
                        ()

                Returns:
                        ()
        '''
        return _mod_pow(x, self.d, self.n)

In [5]:
pve = PairValEnc(17,19)

x = 10
y = _inv_mult(10, pve.n)
print(x,y)

10 97


In [7]:
pve._cmp(x,y)

1

In [8]:
x1 = pve._encrypt(x)
y1 = pve._encrypt(y)

In [9]:
x1

249

In [10]:
y1

48

In [11]:
pve._cmp(x1,y1)

1

## Example for demonstration purposes

Let's say we're writing an algorithm to put patients in touch with healthcare professionals who specialize in treating their infections. Assume that people only want doctors whose experience corresponds exactly to their cases, so as to get the most relevant treatment. Of course, people want their medical history to be kept secret from anyone who is *not* yet their doctor.

For the sake of developing germane values, let us consider a system of 4 variables:
- Patient or doctor?
- Bacterial Infection?
- Fungal Infection?
- Viral Infection?

Each of these variables will be represented by a binary digit. Note that the encoding for doctors is opposite that of patients. Thus, we have the following example encodings of people's healthcare status:
- 1110 – doctor specializing in viral infection
- 0110 – patient with both bacterial and fungal infection
- 0010 – patient with only a fungal infection
- 1111 – doctor who does not specialize in infections
- 0000 – patient who does not have an infection

Note that two people have matching status – indicating they should be paired as doctor and patient – if their bitwise values are different. This means that taking the bitwise exclusive-or operation of their status codes would return $1111_2$, or 15.

Now, we have a mapping between people in our example scenario and X = {0,1,2,...,15}, and a function (bitwise xor) corresponding to the notion of a patient-doctor match. In order to utilize the pair-value encryption scheme, we must map X to Z/nZ, the set of integers modulo n, where n is the PVE modulus. We also require that |X| < phi(n), such that any pair which is equivalent in X/~ can be constructed to map to a pair of inverses modulo n. In this case, we take n = 33.

The below code is used for creating a dictionary mapping X -> Z/33Z:

In [13]:
X = list(range(16)) 
match = lambda x, y: int(x ^ y == 15)

n = 33

In [14]:
# Step 1, find a list of matches in X/~
X_matches = []
x = len(X)
for i in range(x):
    for j in range(i,x):
        if match(i,j):
            X_matches.append((i,j))

In [15]:
print(X_matches)

[(0, 15), (1, 14), (2, 13), (3, 12), (4, 11), (5, 10), (6, 9), (7, 8)]


In [20]:
# Step 2, find |X/~| pairs of modular inverses from Z/33Z
Z_matches = []

i = 1
while len(Z_matches) < len(X_matches):
    j = _inv_mult(i,n)
    
    if j is None:
        i += 1
        continue
        
    Z_matches.append((i,j))
    i += 1

In [21]:
print(Z_matches)

[(1, 1), (2, 17), (4, 25), (5, 20), (7, 19), (8, 29), (10, 10), (13, 28)]


In [22]:
# Step 3, map values over
X_to_Z = {}

for i, (x,y) in enumerate(X_matches):
    X_to_Z[x] = Z_matches[i][0]
    X_to_Z[y] = Z_matches[i][1]

In [23]:
X_to_Z

{0: 1,
 15: 1,
 1: 2,
 14: 17,
 2: 4,
 13: 25,
 3: 5,
 12: 20,
 4: 7,
 11: 19,
 5: 8,
 10: 29,
 6: 10,
 9: 10,
 7: 13,
 8: 28}

Now, with this mapping in hand, we may demonstrate the real power of the PVE system.

Let us assume patient A has a bacterial infection, giving them status code 0100, or 4. Patient A wants to be matched with a doctor specializing in treating bacterial infections.

The system lists 3 doctors: X, Y, and Z, who – unbeknownst to patient A – are specialists in fungal/viral infections, bacterial infections, and cancer (e.g. no specialization with infectious diseases).

With the system initialized and the mapping loaded, patient A can find their encrypted status code, as can all the doctors, from which they may share their statuses, compare, and match patient A with the right doctor.

In [34]:
pve = PairValEnc(3,11) # Set n to 33 – in practice, these primes are chosen to be large and kept incredibly secret
pve.load(X, X_to_Z) # Load the scenario's value mapping into Z/33Z

In [35]:
# A calculating their encrypted status code
a_plain = 0b0100
a_enc = pve.encrypt(a_plain)

In [36]:
# X calculating their encrypted status code
x_plain = 0b1100
x_enc = pve.encrypt(x_plain)

In [37]:
# Y calculating their encrypted status code
y_plain = 0b1011
y_enc = pve.encrypt(y_plain)

In [38]:
# Z calculating their encrypted status code
z_plain = 0b1111
z_enc = pve.encrypt(z_plain)

In [39]:
# A compares their code with that of the other doctors:
doctors = {"X": x_enc, "Y": y_enc, "Z": z_enc}

for doctor, enc in doctors.items():
    if pve.compare(a_enc, enc):
        print(f"Match found with doctor {doctor}")

Match found with doctor Y


At this point, given the simplicity and determinism of this system, both A and Y know respectively that Y is an bacterial infection specialist and A is a patient with a bacterial infection. However, note that this may not generally be the case, especially given a more flexible definition of a matching function.